## 导入包

In [2]:
import torch
# 导入PyTorch库的基本形式，它允许你访问PyTorch库中的所有功能，包括张量操作、自动微分、优化器、模型保存和加载等
import torch.nn as nn
# 更方便地访问构建神经网络所需的类和函数，例如层（如 nn.Linear、nn.Conv2d）、激活函数（如 nn.ReLU）、损失函数（如 nn.CrossEntropyLoss）等
import torch.nn.functional as F
# 它提供了一系列的函数，这些函数可以用于构建神经网络的层和损失函数
from torchinfo import summary
# 用于打印模型的详细摘要。这个摘要包括每一层的输出形状、参数数量、总参数数量以及模型的总计算量

ModuleNotFoundError: No module named 'torchinfo'

## SimpleCNNWithBatchNorm

In [3]:
class SimpleCNNWithBatchNorm(nn.Module):
    def __init__(self):
        super(SimpleCNNWithBatchNorm, self).__init__()
        # 卷积层1: 输入通道1，输出通道32，卷积核大小3x3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # 批量归一化层1
        self.bn1 = nn.BatchNorm2d(32)
        # 卷积层2: 输入通道32，输出通道64，卷积核大小3x3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # 批量归一化层2
        self.bn2 = nn.BatchNorm2d(64)
        # 池化层: 最大池化
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 全连接层
        # 初始化第一个全连接层。这里的输入特征数量是64 * 7 * 7，这是基于假设输入图片大小为28x28，并且经过两次2x2的最大池化后，特征图大小变为7x7。这个全连接层将这些特征连接到128个节点
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 假设输入图片大小为28x28
        # 初始化第二个全连接层，它将128个节点连接到10个输出节点，对应于10个类别
        self.fc2 = nn.Linear(128, 10)  # 假设有10个类别

    def forward(self, x):
        # 通过卷积层1，批量归一化层1，使用ReLU激活函数，然后进行最大池化
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        # 通过卷积层2，批量归一化层2，使用ReLU激活函数，然后进行最大池化
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        # 展平特征图
        x = x.view(-1, 64 * 7 * 7)  # 将多维的特征图x展平成一个一维向量。这里假设经过两次池化后，特征图的大小为7x7，并且有64个通道，因此展平后的向量长度为64 * 7 * 7
        # 通过全连接层1
        x = F.relu(self.fc1(x))
        # 通过丢弃层（Dropout）
        x = F.dropout(x, p=0.5, training=self.training)
        # 通过全连接层2输出最终结果
        x = self.fc2(x)
        return x

In [4]:
# 实例化模型
model = SimpleCNNWithBatchNorm()

# 打印模型结构
print(model)

SimpleCNNWithBatchNorm(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


## onnx

In [5]:
# 设置模型为评估模式
model.eval()

# 创建示例输入
dummy_input = torch.randn(1, 1, 28, 28)

# 导出模型为 ONNX 格式
output_file = "SimpleCNNWithBatchNorm.onnx"
torch.onnx.export(model, dummy_input, output_file,
                  export_params=True,        # 存储训练过的参数
                  opset_version=10,         # ONNX 版本
                  do_constant_folding=True, # 是否执行常量折叠优化
                  input_names=['input'],    # 输入名称
                  output_names=['output'],  # 输出名称
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 批次大小动态
                  )

print(f"ONNX model exported to {output_file}")

OnnxExporterError: Module onnx is not installed!

## netron

In [5]:
import netron

# 使用 netron 查看 ONNX 模型
netron.start(output_file)

Serving 'SimpleCNNWithBatchNorm.onnx' at http://localhost:8080


('localhost', 8080)